In [27]:
"""
Führt Regressionsanalyse durch

"""
import pandas as pd
import numpy as np



#Initmessage
print ("starting Regression") 

#lade von CSV
data_frame_investigated = pd.read_csv("Data/crypto-markets.csv", sep="," )


print("data collected")

print(data_frame_investigated.head())

starting Regression
data collected
      slug symbol     name        date  ranknow    open    high     low  \
0  bitcoin    BTC  Bitcoin  2013-04-28        1  135.30  135.98  132.10   
1  bitcoin    BTC  Bitcoin  2013-04-29        1  134.44  147.49  134.00   
2  bitcoin    BTC  Bitcoin  2013-04-30        1  144.00  146.93  134.05   
3  bitcoin    BTC  Bitcoin  2013-05-01        1  139.00  139.89  107.72   
4  bitcoin    BTC  Bitcoin  2013-05-02        1  116.38  125.60   92.28   

    close  volume        market  close_ratio  spread  
0  134.21     0.0  1.500520e+09       0.5438    3.88  
1  144.54     0.0  1.491160e+09       0.7813   13.49  
2  139.00     0.0  1.597780e+09       0.3843   12.88  
3  116.99     0.0  1.542820e+09       0.2882   32.17  
4  105.21     0.0  1.292190e+09       0.3881   33.32  


In [28]:
#eliminiere redundante Attribute
REDUNDANT_ATTRIBUTES=['symbol','name'] 
data_frame_investigated.drop(REDUNDANT_ATTRIBUTES,axis=1,inplace=True) #loeschen aus Dataframe

#set Date as Index


print(data_frame_investigated.head())


      slug        date  ranknow    open    high     low   close  volume  \
0  bitcoin  2013-04-28        1  135.30  135.98  132.10  134.21     0.0   
1  bitcoin  2013-04-29        1  134.44  147.49  134.00  144.54     0.0   
2  bitcoin  2013-04-30        1  144.00  146.93  134.05  139.00     0.0   
3  bitcoin  2013-05-01        1  139.00  139.89  107.72  116.99     0.0   
4  bitcoin  2013-05-02        1  116.38  125.60   92.28  105.21     0.0   

         market  close_ratio  spread  
0  1.500520e+09       0.5438    3.88  
1  1.491160e+09       0.7813   13.49  
2  1.597780e+09       0.3843   12.88  
3  1.542820e+09       0.2882   32.17  
4  1.292190e+09       0.3881   33.32  


In [52]:

from sklearn.model_selection import train_test_split #library fuer Split
from sklearn import linear_model #lineare Regression
import math #import fuer msqError
from sklearn.metrics import mean_squared_error

#dependency :patsy

def cryptoregress(crypto_id, feature_to_compare,crypto_id_feature_name="slug"):
    """
    lineare Regression zur Bestimmung eines Features anhand gleichen Features der anderen Cryptos
    """
    #Cryptos Gruppieren
    
    #ich kann nur anhand von Tagen schliessen, an denen die Crypto Existiert
    #erstmal alle Tage holen an denen die Crypto gehandelt wird
    dates_of_crypto_trade= data_frame_investigated[
        data_frame_investigated[crypto_id_feature_name] == crypto_id
    ]['date']
    
   
    print(dates_of_crypto_trade.head())
    
    #dann die Tupel Filtern, nur die Tage an denen Crypto getraded wird uebernehmen,
    #nur compare feature uebernehmen
    data_frame_id_exists= data_frame_investigated[
        data_frame_investigated['date'].isin(dates_of_crypto_trade)
    ][['date',crypto_id_feature_name,feature_to_compare]]
    
    #daten Pivotieren so dass Cryptos zu columns werden
    data_frame_id_exists= data_frame_id_exists.pivot(
        index='date', columns=crypto_id_feature_name, values=feature_to_compare
        ).reset_index()
    
 
    
    data_frame_id_exists=data_frame_id_exists.fillna(value=0)
    #data_frame_id_exists=data_frame_id_exists.dropna()
    
   
    #date rausloeschen
    data_frame_id_exists.drop('date',axis=1,inplace=True)
    print("cleaned Frame")
    print(data_frame_id_exists.head())
    
    ###############Train Test Split:
    #Ergebnis vom Features trennen:
    Y = data_frame_id_exists[[crypto_id]]
    data_frame_id_exists.drop(crypto_id,axis=1, inplace = True)
    print("Regress to:")
    print(Y.head())

    
        
    feature_train, feature_test, y_train, y_test = train_test_split (
        data_frame_id_exists,
        Y,
        test_size=0.1
    )

    ##############Regressionsmodell machen
    regression_model = linear_model.LinearRegression(
        fit_intercept=True, normalize=True, copy_X=True, n_jobs=-1)


        #regressionsmodell fitten
    regression_model.fit(feature_train, y_train)
    print("RSquared train:",regression_model.score(feature_train,y_train ))
    print("RSquared test:",regression_model.score(feature_test,y_test ))
    print("MeanSqure train:", math.sqrt( mean_squared_error(regression_model.predict(feature_test),y_test)))
    print("MeanSqure test:", math.sqrt( mean_squared_error(regression_model.predict(feature_train),y_train)))
    
    return regression_model 
       

    


regression_result=cryptoregress('bitcoin','close')

    
    
    

0    2013-04-28
1    2013-04-29
2    2013-04-30
3    2013-05-01
4    2013-05-02
Name: date, dtype: object
cleaned Frame
slug   0x  1337coin  1world  2give  300-token  42-coin  808coin  8bit  \
0     0.0       0.0     0.0    0.0        0.0      0.0      0.0   0.0   
1     0.0       0.0     0.0    0.0        0.0      0.0      0.0   0.0   
2     0.0       0.0     0.0    0.0        0.0      0.0      0.0   0.0   
3     0.0       0.0     0.0    0.0        0.0      0.0      0.0   0.0   
4     0.0       0.0     0.0    0.0        0.0      0.0      0.0   0.0   

slug  abjcoin  abncoin   ...     zeusshield  zilbercoin  zilla  zilliqa  zoin  \
0         0.0      0.0   ...            0.0         0.0    0.0      0.0   0.0   
1         0.0      0.0   ...            0.0         0.0    0.0      0.0   0.0   
2         0.0      0.0   ...            0.0         0.0    0.0      0.0   0.0   
3         0.0      0.0   ...            0.0         0.0    0.0      0.0   0.0   
4         0.0      0.0   ...        

In [30]:
#zeige mir die Koeffizienten
for cryptoiterator, col_name in enumerate(X_train.columns):
    print("The coefficient for {} is {}".format(col_name, regression_result.coef_[0][cryptoiterator]))

NameError: name 'X_train' is not defined

In [ ]:
max(regression_result.coef_)